In [1]:
import submitit
from frust.stepper import Stepper
from pathlib import Path
from tooltoad.chemutils import xyz2mol

# Run single structure

In [2]:
f = Path("../structures/ts4_TMP.xyz")
mols = {}
with open(f, "r") as file:
    xyz_block = file.read()
    mol = xyz2mol(xyz_block)
    mols[f.stem] = (mol, [0])

step = Stepper(list(mols.keys()), step_type="ts1", save_output_dir=False)
df0 = step.build_initial_df(mols)

2025-09-02 14:44:54 INFO  frust.stepper: Working dir: .


In [3]:
def run_orca_calc(
    df,
    results_dir,
    debug=False,
    n_cores=10,
    mem_gb=30,
):
    from pathlib import Path
    import nuse
    nuse.start_monitoring(filter_cgroup=True)

    name = df["custom_name"].iloc[0]

    results_dir = Path(results_dir)
    results_dir.mkdir(exist_ok=True)

    step = Stepper([name],
                step_type="none",
                debug=debug,
                save_output_dir=True,
                save_calc_dirs=False,
                output_base=str(results_dir),
                n_cores=n_cores,
                memory_gb=mem_gb)

    df1 = step.orca(df, "UMA-test-7", {"ExtOpt": None, "NumFreq": None}, uma="omol", xtra_inp_str=
"""%geom
  Calc_Hess  true
  NumHess    true
  Recalc_Hess 5
  MaxIter    300
end""")

    df1.to_parquet(f"{results_dir}/results_{name}.parquet")

In [4]:
DEBUG           = False
N_CORES         = 96
MEM_GB          = 250
TIMEOUT_MIN     = 14400
RESULTS_DIR     = "UMA-test"

executor = submitit.AutoExecutor(f"logs/{RESULTS_DIR}")
executor.update_parameters(
    slurm_partition="kemi1",
    cpus_per_task=N_CORES,
    mem_gb=MEM_GB,
    timeout_min=TIMEOUT_MIN,
    slurm_additional_parameters={"exclude": "node070"},
)

name = df0["custom_name"].iloc[0]
executor.update_parameters(slurm_job_name=name)
executor.submit(run_orca_calc, df0, RESULTS_DIR, DEBUG, N_CORES, MEM_GB)
print(f"Submitted: {name}")

Submitted: ts4_TMP


In [6]:
import pandas as pd

df = pd.read_parquet("UMA-test/results_ts4_TMP.parquet")

step = Stepper(["ts4-freq"],
            step_type="none",
            debug=False,
            save_output_dir=True,
            output_base=str("ts4-dft-freq"),
            n_cores=10,
            memory_gb=35)

df1 = step.orca(df, "DFT-freqs-ts4", options={'wB97X-D3': None, '6-31G**': None, 'TightSCF': None, 'Freq': None})

Attempt 1/5: Encountered error: 'PUGREST.BadRequest: error: '
2025-08-31 20:31:50 INFO  frust.stepper: Working dir: .
2025-08-31 20:31:50 INFO  frust.stepper: [DFT-freqs-ts4-wB97X-D3-6-31G**-Freq] row 0 (ts4_TMP)…


In [17]:
df1

,custom_name,ligand_name,rpos,constraint_atoms,cid,smiles,atoms,coords_embedded,energy_uff,UMA-test-1-OptTS-ExtOpt-electronic_energy,UMA-test-1-OptTS-ExtOpt-normal_termination,UMA-test-1-OptTS-ExtOpt-opt_coords,DFT-freqs-ts4-wB97X-D3-6-31G**-Freq-electronic_energy,DFT-freqs-ts4-wB97X-D3-6-31G**-Freq-gibbs_energy,DFT-freqs-ts4-wB97X-D3-6-31G**-Freq-normal_termination,DFT-freqs-ts4-wB97X-D3-6-31G**-Freq-vibs
0,ts4_TMP,TMP,None,None,0,None,"[C, C, C, C, C, C, H, H, H, H, N, B, H, H, H, ...","[[-2.401391, -1.418555, 1.040286], [-1.131797,...",None,-1629.231343,True,"[[-2.404246, -1.41534, 1.015703], [-1.130448, ...",-1629.042593,-1628.483984,True,"[{'frequency': -142.01, 'mode': [[0.045093, 0...."


In [21]:
df1.to_parquet("UMA-opt-DFT-freqs.parquet")

In [ ]:
step = Stepper(["ts4-freq-UMA"],
            step_type="none",
            debug=False,
            save_output_dir=True,
            output_base=str("ts4-uma-freq"),
            n_cores=10,
            memory_gb=20)

df2 = step.orca(df, "UMA-freq", {"ExtOpt": None, "NumFreq": None}, uma="omol", xtra_inp_str=
"""%geom
  Calc_Hess  true
  NumHess    true
  Recalc_Hess 5
  MaxIter    300
end""")

Attempt 1/5: Encountered error: 'PUGREST.BadRequest: error: '
2025-08-31 21:35:29 INFO  frust.stepper: Working dir: .
2025-08-31 21:35:31 INFO  frust.stepper: [UMA-freq-ExtOpt-NumFreq] row 0 (ts4_TMP)…


In [23]:
df2.to_parquet("UMA-opt-UMA-freqs.parquet")

In [27]:
step = Stepper(["ts4-freq-UMA"],
            step_type="none",
            debug=False,
            save_output_dir=True,
            output_base=str("ts4-uma-irc"),
            n_cores=10,
            memory_gb=30)

df3 = step.orca(df, "UMA-IRC", {"ExtOpt": None, "IRC": None}, uma="omol", save_step=True, xtra_inp_str=
"""%geom
  Calc_Hess  true
  NumHess    true
  MaxIter    300
end""")

Attempt 1/5: Encountered error: 'PUGREST.BadRequest: error: '
2025-08-31 22:07:38 INFO  frust.stepper: Working dir: .
2025-08-31 22:07:40 INFO  frust.stepper: [UMA-IRC-ExtOpt-IRC] row 0 (ts4_TMP)…
